In [1]:
#Python
import os       # Xử lý đường dẫn, file, thư mục
import io       # Xử lý luồng dữ liệu (bytes/streams)
import re       # Regular Expression - tìm kiếm, thay thế theo mẫu
import json     # Đọc / ghi dữ liệu JSON
import base64   # Mã hóa / giải mã base64
import traceback  # Lấy thông tin lỗi khi exception
from datetime import datetime, timedelta 
#GUI - Tkinter
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from tkinter.ttk import Combobox
from PIL import Image, ImageTk # Xử lý ảnh và hiển thị trong Tkinter
#Database
import mysql.connector 
from mysql.connector import errorcode
import google.generativeai as genai 
from app_config import connect_db, get_gemini_model # connect DB, Gemini

/home/bush/.local/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Kết nối MySQL
conn = connect_db()
if conn:
    cursor = conn.cursor()

# Kết nối Gemini
gemini_model = get_gemini_model()

MySQL: Connected successfully
Gemini: Connected successfully (API key valid)


In [3]:
# Chọn Ảnh
def load_image():
    path = filedialog.askopenfilename(filetypes=[("Image files", "*.png *.jpg *.jpeg *.heic")])
    if path:
        try:
            # Kiểm tra kích thước file
            if os.path.getsize(path) > 10 * 1024 * 1024:  # 10MB
                messagebox.showerror("Lỗi", "File ảnh quá lớn (tối đa 10MB).")
                return
            image_path.set(path)
            img = Image.open(path)
            max_width = 600
            width, height = img.size
            aspect_ratio = height / width
            new_width = min(max_width, width)
            new_height = int(new_width * aspect_ratio)
            img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            photo = ImageTk.PhotoImage(img)
            image_label.config(image=photo)
            image_label.image = photo
            # Reset các textbox
            product_name.set("")
            manu_name.set("")
            manu_address.set("")
            manu_phone.set("")
            imp_name.set("")
            imp_address.set("")
            imp_phone.set("")
            mfg_date.set("")
            exp_date.set("")
            product_type.set("")
        except FileNotFoundError:
            messagebox.showerror("Lỗi", "File ảnh không tồn tại.")
        except PIL.UnidentifiedImageError:
            messagebox.showerror("Lỗi", "File không phải định dạng ảnh hợp lệ (PNG, JPG, JPEG, HEIC).")
        except Exception as e:
            messagebox.showerror("Lỗi", f"Lỗi khi tải ảnh: {str(e)}")
            traceback.print_exc()

In [4]:
# Hàm xác thực dữ liệu sản phẩm
def validate_product_data(data, row_index=None):
    error_prefix = f" ở dòng {row_index + 1}" if row_index is not None else ""

    def validate_phone(phone, field_name):
        if not phone or not phone.strip():
            return
        phone_clean = phone.strip()
        phone_digits = re.sub(r'[\s\-\(\)]', '', phone_clean)
        if not re.match(r'^\+?\d{8,15}$', phone_digits):
            raise ValueError(f"Số điện thoại {field_name} không hợp lệ{error_prefix}.\n"
                            f"Định dạng đúng: 0123456789, +84123456789, 0123-456-789, (024) 1234-5678")

    def validate_date(date_str, field_name):
        if date_str and date_str.strip():
            date_clean = date_str.strip()
            date_patterns = ["%d-%m-%Y", "%d/%m/%Y", "%d.%m.%Y"]
            valid_date = False
            parsed_date = None
            for pattern in date_patterns:
                try:
                    parsed_date = datetime.strptime(date_clean, pattern)
                    valid_date = True
                    break
                except ValueError:
                    continue
            if not valid_date:
                raise ValueError(f"Ngày {field_name} không đúng định dạng{error_prefix}.\n"
                                f"Định dạng đúng: DD-MM-YYYY, DD/MM/YYYY hoặc DD.MM.YYYY")
            current_date = datetime.now()
            if parsed_date > current_date + timedelta(days=3650):
                raise ValueError(f"Ngày {field_name} quá xa trong tương lai{error_prefix}.")
            return parsed_date
        return None

    def validate_length(text, field_name, max_length):
        if text and text.strip() and len(text.strip()) > max_length:
            raise ValueError(f"Trường {field_name} vượt quá {max_length} ký tự{error_prefix}.")

    # Validate phone
    validate_phone(data.get("manufacturer_phone", ""), "nhà sản xuất")
    validate_phone(data.get("importer_phone", ""), "nhập khẩu")

    # Validate dates
    mfg_parsed = validate_date(data.get("manufacturing_date", ""), "sản xuất")
    exp_parsed = validate_date(data.get("expiry_date", ""), "hết hạn")

    # Validate logic: expiry_date >= manufacturing_date
    if mfg_parsed and exp_parsed and exp_parsed < mfg_parsed:
        raise ValueError(f"Ngày hết hạn phải lớn hơn hoặc bằng ngày sản xuất{error_prefix}.")

    # Validate text lengths
    validate_length(data.get("product_name", ""), "Tên sản phẩm", 255)
    validate_length(data.get("manufacturer_name", ""), "Tên công ty sản xuất", 255)
    validate_length(data.get("manufacturer_address", ""), "Địa chỉ sản xuất", 500)
    validate_length(data.get("importer_name", ""), "Tên công ty nhập khẩu", 255)
    validate_length(data.get("importer_address", ""), "Địa chỉ nhập khẩu", 500)

    # Validate product_type
    valid_types = ["Bánh kẹo", "Nước uống", "Đông lạnh", "Hàng tiêu dùng", "Điện máy", "Nội thất", "Khác"]
    product_type = data.get("product_type", "")
    if product_type and product_type.strip() and product_type.strip() not in valid_types:
        raise ValueError(f"Loại sản phẩm không hợp lệ{error_prefix}.\nCác loại hợp lệ: {', '.join(valid_types)}")

    # Validate UTF-8 encoding
    try:
        for field, name in [
            (data.get("product_name", ""), "Tên sản phẩm"),
            (data.get("manufacturer_name", ""), "Tên công ty sản xuất"),
            (data.get("manufacturer_address", ""), "Địa chỉ sản xuất"),
            (data.get("importer_name", ""), "Tên công ty nhập khẩu"),
            (data.get("importer_address", ""), "Địa chỉ nhập khẩu")
        ]:
            if field and field.strip():
                field.encode('utf-8')
    except UnicodeEncodeError:
        raise ValueError(f"Một hoặc nhiều trường chứa ký tự không hợp lệ với UTF-8{error_prefix}.")

    # Validate image_base64 if provided
    if data.get("image_base64"):
        try:
            base64.b64decode(data["image_base64"])
        except Exception:
            raise ValueError(f"Dữ liệu image_base64 không hợp lệ{error_prefix}.")

    # Convert dates to MySQL format
    def convert_date_for_mysql(date_str):
        if not date_str or not date_str.strip():
            return None
        date_patterns = ["%d-%m-%Y", "%d/%m/%Y", "%d.%m.%Y"]
        for pattern in date_patterns:
            try:
                parsed_date = datetime.strptime(date_str.strip(), pattern)
                return parsed_date.strftime("%Y-%m-%d")
            except ValueError:
                continue
        return None

    return {
        "mfg_mysql": convert_date_for_mysql(data.get("manufacturing_date", "")),
        "exp_mysql": convert_date_for_mysql(data.get("expiry_date", ""))
    }

In [5]:
# Hàm insert hoặc update dữ liệu vào database
def insert_or_update_product(data, db, cursor):
    # Kiểm tra sản phẩm đã tồn tại dựa trên image_name
    cursor.execute("SELECT id FROM products WHERE image_name = %s", (data.get("image_name"),))
    existing = cursor.fetchone()

    if existing and existing[0]:  # Nếu sản phẩm đã tồn tại, thực hiện UPDATE
        query = """
            UPDATE products SET
                image_path = %s, image_base64 = %s, product_name = %s,
                manufacturer_name = %s, manufacturer_address = %s, manufacturer_phone = %s,
                importer_name = %s, importer_address = %s, importer_phone = %s,
                manufacturing_date = %s, expiry_date = %s, product_type = %s
            WHERE id = %s
        """
        values = (
            data.get("image_path", ""),
            data.get("image_base64", ""),
            data.get("product_name", "").strip() or None,
            data.get("manufacturer_name", "").strip() or None,
            data.get("manufacturer_address", "").strip() or None,
            data.get("manufacturer_phone", "").strip() or None,
            data.get("importer_name", "").strip() or None,
            data.get("importer_address", "").strip() or None,
            data.get("importer_phone", "").strip() or None,
            data.get("mfg_mysql"),
            data.get("exp_mysql"),
            data.get("product_type", "").strip() or None,
            existing[0]
        )
        cursor.execute(query, values)
        return "update", existing[0]
    else:  # Nếu sản phẩm mới, thực hiện INSERT
        query = """
            INSERT INTO products (
                image_name, image_path, image_base64,
                product_name, manufacturer_name, manufacturer_address, manufacturer_phone,
                importer_name, importer_address, importer_phone,
                manufacturing_date, expiry_date, product_type
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = (
            data.get("image_name", ""),
            data.get("image_path", ""),
            data.get("image_base64", ""),
            data.get("product_name", "").strip() or None,
            data.get("manufacturer_name", "").strip() or None,
            data.get("manufacturer_address", "").strip() or None,
            data.get("manufacturer_phone", "").strip() or None,
            data.get("importer_name", "").strip() or None,
            data.get("importer_address", "").strip() or None,
            data.get("importer_phone", "").strip() or None,
            data.get("mfg_mysql"),
            data.get("exp_mysql"),
            data.get("product_type", "").strip() or None
        )
        cursor.execute(query, values)
        return "insert", cursor.lastrowid

In [6]:
# Save - Database
def save_data():
    try:
        # Kiểm tra có ảnh hay không
        if not image_path.get():
            messagebox.showwarning("Lỗi", "Phải chọn ảnh.")
            return

        # Đọc file ảnh
        try:
            with open(image_path.get(), "rb") as img_file:
                encoded_string = base64.b64encode(img_file.read()).decode('utf-8')
        except FileNotFoundError:
            messagebox.showerror("Lỗi", "File ảnh không tồn tại.")
            return
        except Exception as e:
            messagebox.showerror("Lỗi", f"Lỗi đọc file ảnh: {str(e)}")
            return

        # Chuẩn bị dữ liệu
        data = {
            "image_name": os.path.basename(image_path.get()),
            "image_path": image_path.get(),
            "image_base64": encoded_string,
            "product_name": product_name.get(),
            "manufacturer_name": manu_name.get(),
            "manufacturer_address": manu_address.get(),
            "manufacturer_phone": manu_phone.get(),
            "importer_name": imp_name.get(),
            "importer_address": imp_address.get(),
            "importer_phone": imp_phone.get(),
            "manufacturing_date": mfg_date.get(),
            "expiry_date": exp_date.get(),
            "product_type": product_type.get()
        }

        # Validate dữ liệu
        validated_data = validate_product_data(data)
        data.update(validated_data)

        # Kết nối và insert/update vào DB
        db = None
        try:
            print("[DEBUG] Kết nối database...")
            db = connect_db()
            if not db:
                raise Exception("Không thể kết nối đến database.")
            cursor = db.cursor()
            action, record_id = insert_or_update_product(data, db, cursor)
            db.commit()
            messagebox.showinfo("Thành công", 
                               f"Đã {'cập nhật' if action == 'update' else 'lưu'} thông tin sản phẩm (ID: {record_id}).")
            print(f"[DEBUG] {'UPDATE' if action == 'update' else 'INSERT'} thành công với ID: {record_id}")
        except mysql.connector.IntegrityError as e:
            messagebox.showerror("Lỗi DB", "Dữ liệu không hợp lệ.")
            print("[ERROR] IntegrityError:", str(e))
        except mysql.connector.DataError as e:
            messagebox.showerror("Lỗi DB", "Dữ liệu quá dài hoặc không đúng định dạng.")
            print("[ERROR] DataError:", str(e))
        except mysql.connector.OperationalError as e:
            messagebox.showerror("Lỗi DB", "Không thể kết nối database.")
            print("[ERROR] OperationalError:", str(e))
        except Exception as e:
            messagebox.showerror("Lỗi", f"Lỗi không xác định: {str(e)}")
            print("[ERROR]", str(e))
            traceback.print_exc()
        finally:
            if db and db.is_connected():
                cursor.close()
                db.close()
                print("[DEBUG] Đóng kết nối DB.")

    except ValueError as ve:
        messagebox.showerror("Lỗi nhập liệu", str(ve))
        print("[ERROR] Validation:", str(ve))
    except Exception as e:
        messagebox.showerror("Lỗi", f"Lỗi không xác định: {str(e)}")
        print("[ERROR]", str(e))
        traceback.print_exc()

In [7]:
# Import JSON file to Database
def import_json():
    try:
        # Mở dialog để chọn file JSON
        file_path = filedialog.askopenfilename(filetypes=[("JSON files", "*.json")])
        if not file_path:
            return  # Người dùng hủy chọn file

        # Đọc file JSON
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if not isinstance(data, list):
            messagebox.showerror("Lỗi", "File JSON không đúng định dạng: Phải là danh sách các sản phẩm.")
            return

        # Kết nối database
        db = connect_db()
        cursor = db.cursor()
        inserted_count = 0
        updated_count = 0

        try:
            for i, row in enumerate(data):
                # Kiểm tra sự tồn tại của các trường tiềm năng
                potential_fields = ["image_name", "image_path", "image_base64", "product_name", 
                                  "manufacturer", "importer", "manufacturing_date", "expiry_date", "type"]
                missing_fields = [field for field in potential_fields if field not in row]
                if missing_fields:
                    print(f"[DEBUG] Dòng {i + 1}: Thiếu các trường: {', '.join(missing_fields)}. Sẽ sử dụng giá trị mặc định.")

                # Xử lý manufacturer và importer linh hoạt
                manufacturer = row.get("manufacturer", {})
                if "manufacturer" in row and not isinstance(manufacturer, dict):
                    raise ValueError(f"Dòng {i + 1}: Trường manufacturer phải là object.")
                importer = row.get("importer", {})
                if "importer" in row and not isinstance(importer, dict):
                    raise ValueError(f"Dòng {i + 1}: Trường importer phải là object.")

                # Chuẩn bị dữ liệu, sử dụng giá trị mặc định nếu trường bị thiếu hoặc trống
                data_row = {
                    "image_name": row.get("image_name", ""),
                    "image_path": row.get("image_path", ""),
                    "image_base64": row.get("image_base64", ""),
                    "product_name": row.get("product_name", ""),
                    "manufacturer_name": manufacturer.get("company_name", ""),
                    "manufacturer_address": manufacturer.get("address", ""),
                    "manufacturer_phone": manufacturer.get("phone", ""),
                    "importer_name": importer.get("company_name", ""),
                    "importer_address": importer.get("address", ""),
                    "importer_phone": importer.get("phone", ""),
                    "manufacturing_date": row.get("manufacturing_date", ""),
                    "expiry_date": row.get("expiry_date", ""),
                    "product_type": row.get("type", "")
                }

                # Validate dữ liệu
                validated_data = validate_product_data(data_row, row_index=i)
                data_row.update(validated_data)

                # Insert hoặc update vào DB
                try:
                    action, _ = insert_or_update_product(data_row, db, cursor)
                    if action == "insert":
                        inserted_count += 1
                    else:
                        updated_count += 1
                except mysql.connector.IntegrityError:
                    messagebox.showwarning("Cảnh báo", f"Bỏ qua dòng {i + 1}: Dữ liệu không hợp lệ.")
                    continue
                except mysql.connector.DataError:
                    messagebox.showwarning("Cảnh báo", f"Bỏ qua dòng {i + 1}: Dữ liệu quá dài hoặc không đúng định dạng.")
                    continue

            db.commit()
            messagebox.showinfo("Thành công", f"Đã nhập {inserted_count} sản phẩm và cập nhật {updated_count} sản phẩm từ file JSON.")
        except ValueError as ve:
            messagebox.showerror("Lỗi nhập liệu", str(ve))
        except Exception as e:
            messagebox.showerror("Lỗi", f"Lỗi khi nhập JSON: {str(e)}")
            traceback.print_exc()
        finally:
            if db and db.is_connected():
                cursor.close()
                db.close()
                print("[DEBUG] Đóng kết nối DB.")

    except FileNotFoundError:
        messagebox.showerror("Lỗi", "File JSON không tồn tại.")
    except json.JSONDecodeError:
        messagebox.showerror("Lỗi", "File JSON không đúng định dạng.")
    except Exception as e:
        messagebox.showerror("Lỗi", f"Lỗi không xác định khi nhập JSON: {str(e)}")
        traceback.print_exc()

In [8]:
# Xuất Json file 
def export_json():
    try:
        db = connect_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM products")
        rows = cursor.fetchall()
        cursor.close()
        db.close()

        def norm(v):
            if v is None:
                return ""
            return v  # giữ nguyên nếu không phải None
        
        # Chuyển đổi date objects thành string trước khi serialize JSON
        for row in rows:
            # Chuyển đổi date/datetime objects thành string
            for key, value in row.items():
                if hasattr(value, 'strftime'):  # Kiểm tra nếu là date/datetime object
                    if key in ['manufacturing_date', 'expiry_date']:
                        row[key] = value.strftime('%d-%m-%Y')  # Format DD-MM-YYYY
                    else:
                        row[key] = value.strftime('%Y-%m-%d %H:%M:%S')  # Format datetime đầy đủ
            
             # Tái cấu trúc + chuẩn hóa tránh null 
            row["manufacturer"] = {
                "company_name": norm(row.pop("manufacturer_name")),
                "address": norm(row.pop("manufacturer_address")),
                "phone": norm(row.pop("manufacturer_phone"))
            }
            row["importer"] = {
                "company_name": norm(row.pop("importer_name")),
                "address": norm(row.pop("importer_address")),
                "phone": norm(row.pop("importer_phone"))
            }

             # Chuẩn hoá các field còn lại (None -> "")
            for k, v in list(row.items()):
                row[k] = norm(v)
        
         # Tạo thư mục lưu file nếu chưa có
        folder_name = "data_label"
        os.makedirs(folder_name, exist_ok=True)

        # Tạo tên file mới theo thời gian
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_path = os.path.join(folder_name, f"data_label_{timestamp}.json")

        # Ghi ra file mới
        with open(file_path, "w", encoding='utf-8') as f:
            json.dump(rows, f, indent=4, ensure_ascii=False)

        messagebox.showinfo("Xuất file", f"Đã xuất ra file:\n{file_path}")
    except PermissionError:
        messagebox.showerror("Lỗi", "Không thể ghi file JSON (file đang mở hoặc thiếu quyền).")
    except Exception as e:
        messagebox.showerror("Lỗi", f"Lỗi xuất JSON: {str(e)}")
        logging.error(f"Export JSON error: {str(e)}")
        traceback.print_exc()

In [9]:
#Hiển thị cửa sổ danh sách
def show_data_window():
    try:
        db = connect_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("""
            SELECT product_type, COUNT(*) as count 
            FROM products 
            GROUP BY product_type 
            ORDER BY product_type
        """)
        type_counts = cursor.fetchall()
        cursor.close()
        db.close()

        data_window = tk.Toplevel(app)
        data_window.title("Danh sách sản phẩm theo loại")
        data_window.geometry("1200x800")
        data_window.grab_set()  # Giữ focus trên data_window

        main_frame = tk.Frame(data_window)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        left_frame = tk.Frame(main_frame, width=250)
        left_frame.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 10))
        left_frame.pack_propagate(False)

        tk.Label(left_frame, text="Loại sản phẩm", font=("Arial", 14, "bold")).pack(pady=10)
        type_listbox = tk.Listbox(left_frame, font=("Arial", 12))
        type_listbox.pack(fill=tk.BOTH, expand=True, pady=5)

        total_count = sum(item['count'] for item in type_counts)
        type_listbox.insert(0, f"Tất cả ({total_count})")
        for item in type_counts:
            type_listbox.insert(tk.END, f"{item['product_type'] or 'Không phân loại'} ({item['count']})")

        right_frame = tk.Frame(main_frame)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        tree_frame = tk.Frame(right_frame)
        tree_frame.pack(fill=tk.BOTH, expand=True)
        tree_scroll = ttk.Scrollbar(tree_frame)
        tree_scroll.pack(side=tk.RIGHT, fill=tk.Y)

        columns = ("ID", "Tên sản phẩm", "Nhà sản xuất", "Địa chỉ SX", "SĐT SX", "Nhập khẩu", "Địa chỉ NK", "SĐT NK", "Ngày SX", "Ngày HH", "Loại")
        tree = ttk.Treeview(tree_frame, columns=columns, show="headings", yscrollcommand=tree_scroll.set)
        tree_scroll.config(command=tree.yview)

        for col, width in zip(columns, [50, 200, 150, 200, 100, 150, 200, 100, 100, 100, 120]):
            tree.heading(col, text=col)
            tree.column(col, width=width)
        tree.pack(fill=tk.BOTH, expand=True)

        detail_frame = tk.Frame(right_frame)
        detail_frame.pack(fill=tk.X, pady=10)
        info_frame = tk.Frame(detail_frame)
        info_frame.pack(fill=tk.X)

        image_display = tk.Label(info_frame)
        image_display.pack(side=tk.LEFT, padx=10)

        text_info_frame = tk.Frame(info_frame)
        text_info_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10)
        info_text = tk.Text(text_info_frame, height=8, width=50, wrap=tk.WORD)
        info_scrollbar = ttk.Scrollbar(text_info_frame, orient=tk.VERTICAL, command=info_text.yview)
        info_text.configure(yscrollcommand=info_scrollbar.set)
        info_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        info_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Hàm load dữ liệu
        def load_products_by_type(selected_type=None):
            try:
                db = connect_db()
                cursor = db.cursor(dictionary=True)
                if selected_type == "Tất cả" or selected_type is None:
                    cursor.execute("SELECT * FROM products ORDER BY id DESC")
                elif selected_type == "Không phân loại":
                    cursor.execute("SELECT * FROM products WHERE product_type IS NULL ORDER BY id DESC")
                else:
                    cursor.execute("SELECT * FROM products WHERE product_type = %s ORDER BY id DESC", (selected_type,))
                products = cursor.fetchall()
                cursor.close()
                db.close()

                tree.delete(*tree.get_children())
                for p in products:
                    tree.insert("", tk.END, values=(
                        p['id'],
                        p['product_name'] or "",
                        p['manufacturer_name'] or "",
                        p['manufacturer_address'] or "",
                        p['manufacturer_phone'] or "",
                        p['importer_name'] or "",
                        p['importer_address'] or "",
                        p['importer_phone'] or "",
                        p['manufacturing_date'].strftime('%d-%m-%Y') if p['manufacturing_date'] else "",
                        p['expiry_date'].strftime('%d-%m-%Y') if p['expiry_date'] else "",
                        p['product_type'] or "Không phân loại"
                    ))
                tree.products_data = products
            except Exception as e:
                messagebox.showerror("Lỗi", f"Không thể tải dữ liệu: {str(e)}", parent=data_window)
                logging.error(f"Load data error: {str(e)}")

        def on_type_select(event):
            selection = type_listbox.curselection()
            if selection:
                selected_type = type_listbox.get(selection[0]).split(" (")[0]
                load_products_by_type(selected_type)

        def on_product_select(event):
            selection = tree.selection()
            if selection and hasattr(tree, 'products_data'):
                pid = tree.item(selection[0])['values'][0]
                product = next((p for p in tree.products_data if p['id'] == pid), None)
                if product:
                    info_text.delete(1.0, tk.END)
                    info_text.insert(1.0, 
                        f"Tên sản phẩm: {product['product_name'] or 'N/A'}\n"
                        f"Nhà sản xuất: {product['manufacturer_name'] or 'N/A'}\n"
                        f"Địa chỉ SX: {product['manufacturer_address'] or 'N/A'}\n"
                        f"SĐT SX: {product['manufacturer_phone'] or 'N/A'}\n"
                        f"Nhập khẩu: {product['importer_name'] or 'N/A'}\n"
                        f"Địa chỉ NK: {product['importer_address'] or 'N/A'}\n"
                        f"SĐT NK: {product['importer_phone'] or 'N/A'}\n"
                        f"Ngày SX: {product['manufacturing_date'].strftime('%d-%m-%Y') if product['manufacturing_date'] else 'N/A'}\n"
                        f"Ngày HH: {product['expiry_date'].strftime('%d-%m-%Y') if product['expiry_date'] else 'N/A'}\n"
                        f"Loại: {product['product_type'] or 'Không phân loại'}"
                    )
                    if product['image_base64']:
                        img_data = base64.b64decode(product['image_base64'])
                        img = Image.open(io.BytesIO(img_data))
                        img.thumbnail((200, 200))
                        photo = ImageTk.PhotoImage(img)
                        image_display.config(image=photo)
                        image_display.image = photo

        # Hàm xóa sản phẩm
        def delete_selected_product():
            selection = tree.selection()
            if not selection:
                messagebox.showwarning("Cảnh báo", "Vui lòng chọn sản phẩm để xóa.", parent=data_window)
                return
            pid = tree.item(selection[0])['values'][0]
            confirm = messagebox.askyesno(
                "Xác nhận",
                f"Bạn có chắc muốn xóa sản phẩm ID {pid}?",
                parent=data_window
            )
            if confirm:
                try:
                    db = connect_db()
                    cursor = db.cursor()
                    cursor.execute("DELETE FROM products WHERE id = %s", (pid,))
                    db.commit()
                    cursor.close()
                    db.close()
                    messagebox.showinfo("Thành công", f"Đã xóa sản phẩm ID {pid}", parent=data_window)
                    load_products_by_type("Tất cả")
                except Exception as e:
                    messagebox.showerror("Lỗi", f"Không thể xóa: {str(e)}", parent=data_window)
                    logging.error(f"Delete product error: {str(e)}")

        # Hàm sửa sản phẩm
        def edit_selected_product():
            selection = tree.selection()
            if not selection:
                messagebox.showwarning("Cảnh báo", "Vui lòng chọn sản phẩm để sửa.", parent=data_window)
                return
            pid = tree.item(selection[0])['values'][0]
            product = next((p for p in tree.products_data if p['id'] == pid), None)
            if product:
                # Điền thông tin vào form chính
                image_path.set(product['image_path'] or "")
                product_name.set(product['product_name'] or "")
                manu_name.set(product['manufacturer_name'] or "")
                manu_address.set(product['manufacturer_address'] or "")
                manu_phone.set(product['manufacturer_phone'] or "")
                imp_name.set(product['importer_name'] or "")
                imp_address.set(product['importer_address'] or "")
                imp_phone.set(product['importer_phone'] or "")
                mfg_date.set(product['manufacturing_date'].strftime('%d-%m-%Y') if product['manufacturing_date'] else "")
                exp_date.set(product['expiry_date'].strftime('%d-%m-%Y') if product['expiry_date'] else "")
                product_type.set(product['product_type'] or "")
                # Load ảnh
                if product['image_base64']:
                    img_data = base64.b64decode(product['image_base64'])
                    img = Image.open(io.BytesIO(img_data))
                    max_width = 600
                    width, height = img.size
                    aspect_ratio = height / width
                    new_width = min(max_width, width)
                    new_height = int(new_width * aspect_ratio)
                    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                    photo = ImageTk.PhotoImage(img)
                    image_label.config(image=photo)
                    image_label.image = photo
                messagebox.showinfo("Sửa", "Thông tin sản phẩm đã được điền vào form. Sửa và lưu lại.", parent=data_window)
                data_window.destroy()  # Đóng cửa sổ sau khi load

        type_listbox.bind('<<ListboxSelect>>', on_type_select)
        tree.bind('<<TreeviewSelect>>', on_product_select)

        # Nút chức năng
        btn_frame = tk.Frame(right_frame)
        btn_frame.pack(pady=5)
        tk.Button(btn_frame, text="🔄 Làm mới", command=lambda: load_products_by_type("Tất cả"), font=("Arial", 12)).pack(side=tk.LEFT, padx=5)
        tk.Button(btn_frame, text="🗑 Xóa", command=delete_selected_product, font=("Arial", 12), fg="red").pack(side=tk.LEFT, padx=5)
        tk.Button(btn_frame, text="✏️ Sửa", command=edit_selected_product, font=("Arial", 12), fg="blue").pack(side=tk.LEFT, padx=5)

        load_products_by_type("Tất cả")

    except Exception as e:
        messagebox.showerror("Lỗi", f"Không thể mở cửa sổ dữ liệu: {str(e)}", parent=data_window)
        logging.error(f"Show data window error: {str(e)}")

In [10]:
# GỢI Ý TỪ GEMINI (TỰ ĐỘNG ĐIỀN Ô)
def suggest_from_image():
    try:
        if not image_path.get():
            messagebox.showwarning("Lỗi", "Vui lòng chọn ảnh trước.")
            return

        img = Image.open(image_path.get())
        response = gemini_model.generate_content([
            "Dựa vào nhãn sản phẩm trong hình ảnh, hãy trích xuất các thông tin sau theo định dạng:\n"
            "Tên sản phẩm: [tên sản phẩm]\n"
            "Tên công ty: [tên công ty sản xuất]\n"
            "Địa chỉ: [địa chỉ sản xuất]\n"
            "Điện thoại: [số điện thoại sản xuất]\n"
            "Công ty nhập khẩu: [tên công ty nhập khẩu]\n"
            "Địa chỉ nhập khẩu: [địa chỉ nhập khẩu]\n"
            "Điện thoại nhập khẩu: [số điện thoại nhập khẩu]\n"
            "NSX: [ngày sản xuất DD-MM-YYYY]\n"
            "HSD: [hạn sử dụng DD-MM-YYYY]\n"
            "Loại sản phẩm: [loại sản phẩm] chỉ bao gồm Bánh kẹo, Nước uống, Đông lạnh, Hàng tiêu dùng, Điện máy, Nội thất, Khác,"
            "Không có, không biết thì để null",
            img
        ])

        result = response.text

        # Trích xuất thông tin với nhiều pattern
        def extract(patterns, text, default="", is_date=False):
            if isinstance(patterns, str):
                patterns = [patterns]
            
            for pattern in patterns:
                match = re.search(pattern, text, re.IGNORECASE | re.MULTILINE)
                if match:
                    value = match.group(1).strip()

                    # Lọc các giá trị không hợp lệ
                    if value.lower() in ["không có", "không biết", "n/a", "null", "-"]:
                        return default

                    if is_date:
                        # Chuẩn hóa và kiểm tra định dạng ngày
                        date_formats = ["%d-%m-%Y", "%Y-%m-%d", "%d/%m/%Y", "%Y/%m/%d"]
                        for fmt in date_formats:
                            try:
                                # Nếu parse thành công → trả về theo dạng DD-MM-YYYY
                                return datetime.strptime(value, fmt).strftime("%d-%m-%Y")
                            except ValueError:
                                continue
                        # Nếu không parse được → trả về mặc định
                        return default

                    return value
            return default

        # Trích xuất với nhiều pattern cho mỗi trường
        product_name.set(extract([
            r"Tên sản phẩm:\s*(.*?)(?:\n|$)",
            r"Product name:\s*(.*?)(?:\n|$)",
            r"Sản phẩm:\s*(.*?)(?:\n|$)"
        ], result))

        manu_name.set(extract([
            r"Tên công ty:\s*(.*?)(?:\n|$)",
            r"Công ty sản xuất:\s*(.*?)(?:\n|$)",
            r"Nhà sản xuất:\s*(.*?)(?:\n|$)",
            r"Manufacturer:\s*(.*?)(?:\n|$)"
        ], result))

        manu_address.set(extract([
            r"Địa chỉ:\s*(.*?)(?:\n|$)",
            r"Địa chỉ sản xuất:\s*(.*?)(?:\n|$)",
            r"Address:\s*(.*?)(?:\n|$)"
        ], result))

        manu_phone.set(extract([
            r"Điện thoại:\s*(.*?)(?:\n|$)",
            r"Phone:\s*(.*?)(?:\n|$)",
            r"Tel:\s*(.*?)(?:\n|$)",
            r"ĐT:\s*(.*?)(?:\n|$)"
        ], result))

        imp_name.set(extract([
            r"Công ty nhập khẩu:\s*(.*?)(?:\n|$)",
            r"Nhập khẩu bởi:\s*(.*?)(?:\n|$)",
            r"Importer:\s*(.*?)(?:\n|$)"
        ], result))

        imp_address.set(extract([
            r"Địa chỉ nhập khẩu:\s*(.*?)(?:\n|$)",
            r"Importer address:\s*(.*?)(?:\n|$)"
        ], result))

        imp_phone.set(extract([
            r"Điện thoại nhập khẩu:\s*(.*?)(?:\n|$)",
            r"Importer phone:\s*(.*?)(?:\n|$)"
        ], result))

        mfg_date.set(extract([
            r"NSX:\s*(.*?)(?:\n|$)",
            r"Ngày sản xuất:\s*(.*?)(?:\n|$)",
            r"Manufacturing date:\s*(.*?)(?:\n|$)",
            r"MFG:\s*(.*?)(?:\n|$)"
        ], result))

        exp_date.set(extract([
            r"HSD:\s*(.*?)(?:\n|$)",
            r"Hạn sử dụng:\s*(.*?)(?:\n|$)",
            r"Expiry date:\s*(.*?)(?:\n|$)",
            r"EXP:\s*(.*?)(?:\n|$)"
        ], result))

        product_type.set(extract([
            r"Loại sản phẩm:\s*(.*?)(?:\n|$)",
            r"Category:\s*(.*?)(?:\n|$)",
            r"Type:\s*(.*?)(?:\n|$)"
        ], result))

        messagebox.showinfo("Hoàn thành", "Đã phân tích ảnh và điền thông tin tự động!")

    except Exception as e:
        messagebox.showerror("Lỗi khi gọi Gemini", f"Không thể phân tích ảnh: {str(e)}")

In [11]:
# GUI 
app = tk.Tk()
app.title("Gán nhãn sản phẩm")

# Đặt kích thước lớn ban đầu (80% kích thước màn hình)
screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()
app.geometry(f"{int(screen_width * 0.8)}x{int(screen_height * 0.8)}")

# Ngăn resize nhỏ hơn kích thước đã set
app.minsize(int(screen_width * 0.85), int(screen_height * 0.8))

# Biến 
image_path = tk.StringVar()
product_name = tk.StringVar()
manu_name = tk.StringVar()
manu_address = tk.StringVar()
manu_phone = tk.StringVar()
imp_name = tk.StringVar()
imp_address = tk.StringVar()
imp_phone = tk.StringVar()
mfg_date = tk.StringVar()
exp_date = tk.StringVar()
product_type = tk.StringVar()

fields = [
    ("Tên sản phẩm", product_name),
    ("Tên công ty sản xuất", manu_name),
    ("Địa chỉ nhà sản xuất", manu_address),
    ("Điện thoại nhà sản xuất", manu_phone),
    ("Tên công ty nhập khẩu", imp_name),
    ("Địa chỉ nhà nhập khẩu", imp_address),
    ("Điện thoại nhà nhập khẩu", imp_phone),
    ("Ngày sản xuất (DD-MM-YYYY)", mfg_date),
    ("Ngày hết hạn (DD-MM-YYYY)", exp_date),
]

# Giao diện 
main_frame = tk.Frame(app)
main_frame.pack(fill=tk.BOTH, expand=True, padx=50)  

# KHUNG TRÁI: ẢNH
left_frame = tk.Frame(main_frame)
left_frame.pack(side=tk.LEFT, padx=20, pady=20)


tk.Button(left_frame, text="🖼️ Chọn ảnh", command=load_image, font=("Arial", 14)).pack(pady=10)
tk.Button(left_frame, text="✨ Gợi ý từ Gemini AI", command=suggest_from_image, font=("Arial", 14)).pack(pady=10)

image_label = tk.Label(left_frame)
image_label.pack()

# KHUNG PHẢI: FORM
right_frame = tk.Frame(main_frame)
right_frame.pack(side=tk.RIGHT, padx=50, pady=20, fill=tk.BOTH, expand=True) 

LABEL_FONT = ("Arial", 14)
ENTRY_WIDTH = 50

# Sử dụng grid để căn chỉnh label và entry
for i, (label_text, var) in enumerate(fields):
    tk.Label(right_frame, text=label_text, font=LABEL_FONT).grid(row=i, column=0, sticky="e", padx=(0, 10))
    tk.Entry(right_frame, textvariable=var, width=ENTRY_WIDTH, font=("Arial", 12)).grid(row=i, column=1, sticky="w", pady=10)

# Loại sản phẩm
tk.Label(right_frame, text="Loại sản phẩm", font=LABEL_FONT).grid(row=len(fields), column=0, sticky="e", padx=(0, 10))
product_type_combo = Combobox(right_frame, textvariable=product_type, values=[
    "Bánh kẹo", "Nước uống", "Đông lạnh", "Hàng tiêu dùng", "Điện máy", "Nội thất", "Khác"
], font=("Arial", 12), width=ENTRY_WIDTH)
product_type_combo.grid(row=len(fields), column=1, sticky="w", pady=10)

# Nút
tk.Button(right_frame, text="💾 Lưu vào DB", command=save_data, font=("Arial", 14)).grid(row=len(fields) + 1, column=0, columnspan=2, pady=15)
tk.Button(right_frame, text="📋 Xem dữ liệu", command=show_data_window, font=("Arial", 14)).grid(row=len(fields) + 3, column=0, columnspan=2, pady=15)
tk.Button(right_frame, text="📥 Nhập JSON", command=import_json, font=("Arial", 14)).grid(row=len(fields) + 4, column=0, columnspan=2, pady=15)
tk.Button(right_frame, text="📄 Xuất JSON", command=export_json, font=("Arial", 14)).grid(row=len(fields) + 2, column=0, columnspan=2, pady=15)

#Run App
app.mainloop()